In [2]:
using Pkg
Pkg.activate("Phase2_projet")
Pkg.add(["LinearAlgebra", "SparseArrays", "Krylov", "BenchmarkTools", "SuiteSparseMatrixCollection"])
using LinearAlgebra, SparseArrays, Krylov, BenchmarkTools, SuiteSparseMatrixCollection

  Activating project at `c:\Users\ulric\Projet_MTH8211\Phase2_projet`
   Resolving package versions...
  No Changes to `C:\Users\ulric\Projet_MTH8211\Phase2_projet\Project.toml`
  No Changes to `C:\Users\ulric\Projet_MTH8211\Phase2_projet\Manifest.toml`


In [3]:
function LSRN(A, b; γ=2.0, tol=1e-12, maxiter=500)
    m, n = size(A)
    l = ceil(Int, γ * min(m, n))

    if m >= n
        G = randn(l, m) 
        B = G * A 

        F = qr(B, Val(true))
        R = F.R

        M = inv(R)
        A_precond = A * M
        result = lsqr(A_precond, b; atol=tol, btol=tol, itmax=maxiter)
        x = M * result[1]

    else
        G = randn(l, n)
        B = A * G'

        F = qr(B, Val(true))
        R = F.R

        M = inv(R)
        A_precond = M' * A
        b_precond = M' * b
        result = lsqr(A_precond, b_precond; atol=tol, btol=tol, itmax=maxiter)
        x = result[1]
    end

    return x
end

LSRN (generic function with 1 method)

In [5]:
m, n = 1000000, 1000
A_dense = randn(m, n)
b_dense = randn(m)

1000000-element Vector{Float64}:
  0.5518092654139687
 -1.331605506107442
  1.4850393567144768
  0.2461245300175366
 -0.40841880108489476
 -0.2541939338789641
  0.6775759354785287
  0.6297045323849735
 -2.2511068074838056
  1.2241502504694086
  ⋮
 -0.738508687073445
  0.7914936054199732
  0.3347639909014591
 -0.33305818359196304
  0.7797895463163146
  0.766377776611955
 -0.5502177028486346
 -0.5308026510331171
  0.2035859345484013

In [6]:
x_dense = LSRN(A_dense, b_dense)

residual_dense = norm(A_dense * x_dense - b_dense) / norm(b_dense)
println("Résiduelle relative (dense) : ", residual_dense)
@benchmark norm(A_dense * x_dense - b_dense) / norm(b_dense)

Résiduelle relative (dense) : 0.9995072739738944


BenchmarkTools.Trial: 42 samples with 1 evaluation per sample.
 Range (min … max):  111.887 ms … 130.709 ms  ┊ GC (min … max): 0.00% … 13.48%
 Time  (median):     118.669 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   118.856 ms ±   4.071 ms  ┊ GC (mean ± σ):  0.35% ±  2.08%

         ▁▁      ▁ ▄   ██▁  █ ▁    ▁                             
  ▆▁▁▁▆▆▆██▁▁▁▆▆▆█▁█▁▁▆███▁▆█▆█▁▁▁▁█▁▁▁▁▁▁▁▁▆▁▆▁▁▆▁▁▁▁▁▁▆▁▁▁▁▁▆ ▁
  112 ms           Histogram: frequency by time          131 ms <

 Memory estimate: 15.26 MiB, allocs estimate: 9.

In [6]:
println("Nombre de threads actuellement activés : ", Threads.nthreads())


Nombre de threads actuellement activés : 1


In [5]:
Sys.CPU_THREADS

32

In [7]:
Pkg.add("MatrixMarket")
using MatrixMarket

   Resolving package versions...
  No Changes to `C:\Users\ulric\Projet_MTH8211\Phase2_projet\Project.toml`
  No Changes to `C:\Users\ulric\Projet_MTH8211\Phase2_projet\Manifest.toml`


In [8]:
using LinearAlgebra, SparseArrays, Krylov, Random

In [1]:
matrix_name = "Ill_Stokes"
ssmc = ssmc_db()
pb = ssmc_matrices(ssmc, "", matrix_name)
fetch_ssmc(pb, format="MM")
pb_path = fetch_ssmc(pb, format="MM")
path_mtx = pb_path[1]
A = MatrixMarket.mmread(joinpath(path_mtx, matrix_name * ".mtx"));

UndefVarError: UndefVarError: `ssmc_db` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [9]:
function badly_conditioned_rectangular_matrix(m, n, kappa)
    # Crée deux matrices orthogonales (par QR)
    U, _ = qr(randn(m, n))
    V, _ = qr(randn(n, n))
    # Valeurs singulières décroissantes : de 1 à 1/kappa
    s = range(1.0, 1.0/kappa, length=n)
    S = Diagonal(s)
    A = U * S * V'
    return Matrix(A)
end

badly_conditioned_rectangular_matrix (generic function with 1 method)

In [19]:
m = 100000
n = 1000
kappa = 1e4
A = badly_conditioned_rectangular_matrix(m, n, kappa)
x = randn(n)
b = A * x

100000-element Vector{Float64}:
 -0.0022069025718979146
 -0.08041589558137963
 -0.009935669284809977
  0.008714032646609478
 -0.03376369942922331
 -0.037673745881073764
  0.07312980570229634
 -0.029047427752332895
 -0.02679194453462201
 -0.10092243868326434
  ⋮
  0.08275861522380402
  0.08126312994771755
 -0.0833978497076696
  0.02215145202673785
  0.02196005919124894
  0.06616497383546587
  0.03513834298211814
  0.0006473411808994192
  0.08384294850857575

In [20]:
println("Conditionnement de A :", cond(A))

Conditionnement de A :10000.000000001144


In [ ]:
function lsrn_lsqr(A, b; gamma=2.0, tol=1e-10, itmax=2000)
    m, n = size(A)
    s = ceil(Int, gamma * n) 

    G = randn(s, m)

    Ã = G * A

    Ũ, Σ̃, Ṽ = svd(Ã; full=false)
    r = sum(Σ̃ .> 1e-12)

    Σinv = Diagonal(1.0 ./ Σ̃[1:r])
    V_r = Ṽ[:,1:r]
    N = V_r * Σinv

    AN = A * N
    ŷ, histo = lsqr(AN, b; atol=tol, btol=tol, itmax=itmax, history=true)

    x̂ = N * ŷ
    return x̂, histo
end

lsrn_lsqr (generic function with 1 method)

In [33]:
function lsrn_cg(A, b; gamma=2.0, tol=1e-10, itmax=2000)
    m, n = size(A)
    s = ceil(Int, gamma * n)
    G = randn(s, m)
    Ã = G * A

    # SVD compacte
    Ũ, Σ̃, Ṽ = svd(Ã; full=false)
    r = sum(Σ̃ .> 1e-12)
    Σinv = Diagonal(1.0 ./ Σ̃[1:r])
    V_r = Ṽ[:,1:r]
    N = V_r * Σinv

    # Forme normale
    AN = A * N
    AtAN = AN' * AN
    AtbN = AN' * b

    # Résoudre par CG
    ŷ, histo = cg(AtAN, AtbN; atol=tol, rtol=tol, itmax=itmax, history=true)

    # Reconstruire la solution min-norme
    x̂ = N * ŷ
    return x̂, histo
end

lsrn_cg (generic function with 1 method)

In [ ]:

# LSQR 
println("\n--- LSQR sans préconditionneur ---")
res1, hist1 = lsqr(A, b; atol=1e-10, btol=1e-10, itmax=2000, history=true)
println("Itérations LSQR (sans préc)      : ", hist1.niter)
println("Résidu final LSQR (sans préc)    : ", norm(A * res1 - b) / norm(b))

# CG 
println("\n--- CG sur système normal (sans préc) ---")
AtA = A' * A
Atb = A' * b
res2, hist2 = cg(AtA, Atb; atol=1e-10, rtol=1e-10, itmax=2000, history=true)
println("Itérations CG (sans préc)        : ", hist2.niter)
println("Résidu final CG (sans préc)      : ", norm(AtA * res2 - Atb) / norm(Atb))

#LSRN
println("\n--- Construction du préconditionneur LSRN ---")
γ = 3.0
l = ceil(Int, γ * min(m, n))
G = randn(l, m)
B = G * A
F = qr(B, Val(true))
R = F.R
M = inv(R)

#lsqr-lsrn
println("\n--- LSQR avec préconditionneur LSRN ---")
x_prec, hist3 = lsrn_lsqr(A, b; gamma=2.0, tol=1e-10, itmax=2000)
println("Itérations LSQR (LSRN)           : ", hist3.niter)
println("Résidu final LSQR (LSRN)         : ", norm(A * x_prec - b) / norm(b))

#cg-lsrn
println("\n--- CG sur système normal préconditionné (LSRN) ---")
AtA_prec = A_prec' * A_prec
Atb_prec = A_prec' * b
#res4, hist4 = cg(AtA_prec, Atb_prec; atol=1e-10, rtol=1e-10, itmax=2000, history=true)
x_cg_prec, hist4 = lsrn_cg(A, b; gamma=2.0, tol=1e-10, itmax=2000)
println("Itérations CG (LSRN)             : ", hist4.niter)
println("Résidu final CG (LSRN)           : ", norm(A * x_cg_prec - b) / norm(b))

println("\n--- Résumé du nombre d'itérations ---")
println("LSQR sans préc           : $(hist1.niter)")
println("LSQR avec préc LSRN      : $(hist3.niter)")
println("CG sans préc             : $(hist2.niter)")
println("CG avec préc LSRN        : $(hist4.niter)")



--- LSQR sans préconditionneur ---
Itérations LSQR (sans préc)      : 1509
Résidu final LSQR (sans préc)    : 5.482107284955318e-6

--- CG sur système normal (sans préc) ---
Itérations CG (sans préc)        : 1547
Résidu final CG (sans préc)      : 9.835692754484811e-11

--- Construction du préconditionneur LSRN ---

--- LSQR avec préconditionneur LSRN ---
Itérations LSQR (LSRN)           : 42
Résidu final LSQR (LSRN)         : 2.2110965807978267e-7

--- CG sur système normal préconditionné (LSRN) ---
Itérations CG (LSRN)             : 60
Résidu final CG (LSRN)           : 2.3135340989337673e-10

--- Résumé du nombre d'itérations ---
LSQR sans préc           : 1509
LSQR avec préc LSRN      : 42
CG sans préc             : 1547
CG avec préc LSRN        : 60
